# Lab 5: Object-Oriented Python

## Overview

After Tuesday's lecture, which mostly covered rules, definitions, and semantics, we'll be playing around with actual classes today, writing a fair chunk of code and building several classes to solve a variety of problems.

Recall our starting definitions:

- An *object* has identity
- A *name* is a reference to an object
- A *namespace* is an associative mapping from names to objects
- An *attribute* is any name following a dot ('.')

## Stanford Courses

### Basic Class

Let’s create a class to represent courses at Stanford! A course will have three attributes to start: a department (like `"CS"`), a course code (like `"41"` or `"92SI"`), and a title (like `"hap.py code"`).

```Python
class StanfordCourse:
    def __init__(self, department, code, title):
        self.department = department
        self.code = code
        self.title = title
```

You can assume that all arguments to this constructor will be strings.

Running the following code cell will create a class object `StanfordCourse` and print some information about it.

*Note: If you change the content of this class definition, you will need to re-execute the code cell for it to have any effect. Any instance objects of the old class object will not be automatically updated, so you may need to rerun instantiations of this class object as well.*

In [1]:
class StanfordCourse:
    def __init__(self, department, code, title):
        self.department = department
        self.code = code
        self.title = title
        
print(StanfordCourse)
print(StanfordCourse.mro())
print(StanfordCourse.__init__)

<class '__main__.StanfordCourse'>
[<class '__main__.StanfordCourse'>, <class 'object'>]
<function StanfordCourse.__init__ at 0x000002755967C488>


We create an instance of the class by instantiating the class object, supplying some arguments.

```Python
stanford_python = StanfordCourse("CS", "41", "hap.py code: the python programming language")
```

Print out the three attributes of the `stanford_python` instance object.

In [2]:
class StanfordCourse:
    def __init__(self, department, code, title, students={}):
        self.department = department
        self.code = code
        self.title = title
        self.students = students

    def mark_attendance(self, *students):
        for a in students:
            self.students[a] = "Present"

    def is_present(self, student):
        key, value = student, "Present"
        return key in self.students and value == self.students[key]


stanford_python = StanfordCourse("CS", "41", "hap.py code: The python programming language")

print(stanford_python.title)
print(stanford_python.code)
print( stanford_python.department)

hap.py code: The python programming language
41
CS


### Inheritance

Let's explore inheritance by creating a `StanfordCSCourse` class that takes an additional parameter `recorded` that defaults to `False`.

In [3]:
class StanfordCSCourse(StanfordCourse):
    def __init__(self, department, code, title, recorded=False):
        super().__init__(department, code, title)
        self.is_recorded = recorded

We haven't seen the `super()` call yet, and it's mostly just magic, but it concretely lets us treat the `self` object as an instance object of the immediate superclass (as measured by MRO), so we can call the superclass's `__init__` method.

We can instantiate our new class:

```Python
a = StanfordCourse("CS", "106A", "Programming Methodology")
b = StanfordCSCourse("CS", "106B", "Programming Abstractions")
x = StanfordCSCourse("CS", "106X", "Programming Abstractions", recorded=True)
print(a.code)  # => "106A"
print(b.code)  # => "106B"
```

Read through the following statements and try to predict their output.

```Python
type(a)
isinstance(a, StanfordCourse)
isinstance(b, StanfordCourse)
isinstance(x, StanfordCourse)
isinstance(x, StanfordCSCourse)
issubclass(x, StanfordCSCourse)
issubclass(StanfordCourse, StanfordCSCourse)
type(a) == type(b)
type(b) == type(x)
a == b
b == x
```

In [4]:
a = StanfordCourse("CS", "106A", "Programming Methodology")
b = StanfordCSCourse("CS", "106B", "Programming Abstractions")
x = StanfordCSCourse("CS", "106X", "Programming Abstractions", recorded=True)

print(type(a))
print(isinstance(a, StanfordCourse))
print(isinstance(b, StanfordCourse))
print(isinstance(x, StanfordCourse))
print(isinstance(x, StanfordCSCourse))
print(issubclass(StanfordCourse, StanfordCSCourse))
print(type(a) == type(b))
print(type(b) == type(x))
print(a == b)
print(b == x)

<class '__main__.StanfordCourse'>
True
True
True
True
False
False
True
False
False


In [5]:
print(issubclass(StanfordCSCourse , StanfordCourse))

True


### Additional Attributes

Let's add more functionality to the `StanfordCourse` class!

* Add a attribute `students` to the instances of the `StanfordCourse` class that tracks whether students are present. Initially, students should be an empty set.
* Create a method `mark_attendance(*students)` that takes a variadic number of `students` and marks them as present.
* Create a method `is_present(student)` that takes a student’s name as a parameter and returns `True` if the student is present and `False` otherwise.

In [7]:
class StanfordCourse:
    def __init__(self, department, code, title, students={}):
        self.department = department
        self.code = code
        self.title = title
        self.students = students

    def mark_attendance(self, *students):
        for a in students:
            self.students[a] = "Present"

    def is_present(self, student):
        key, value = student, "Present"
        return key in self.students and value == self.students[key]


stanford_python = StanfordCourse("CS", "41", "hap.py code: The python programming language")

print(stanford_python.title)
print(stanford_python.code)

c = StanfordCourse("CS", "106A", "Programming Methodology", {"Adam": "Present", "Daniel": "Not present"})
print("Is Daniel present? ", c.is_present("Daniel"))
c.mark_attendance("Daniel")
print("Is Daniel present? ", c.is_present("Daniel"))

hap.py code: The python programming language
41
Is Daniel present?  False
Is Daniel present?  True


### Implementing Prerequisites

Now, we'll focus on `StanfordCSCourse`. We want to implement functionality to determine if one computer science course is a prerequisite of another. In our implementation, we will assume that the ordering for courses is determined first by the numeric part of the course code: for example, `140` comes before `255`. If there is a tie, the ordering is determined by the default string ordering of the letters that follow. For example, `106A < 106B`. After implementing, you should be able to see:

```Python
>>> cs106a = StanfordCourse("CS", "106A", "Programming Methodology")
>>> cs106b = StanfordCSCourse("CS", "106B", "Programming Abstractions")
>>> cs107 = StanfordCSCourse("CS", "107", "Computer Organzation and Systems")
>>> cs110 = StanfordCSCourse("CS", "110", "Principles of Computer Systems")
>>> cs110 > cs106b
True
>>> cs107 > cs110
False
```

To accomplish this, you will need to implement a magic method `__le__` that will add functionality to determine if a course is a prerequisite for another course. Read up on [total ordering](https://docs.python.org/3/library/functools.html#functools.total_ordering) to figure out what `__le__` should return based on the argument you pass in.

To give a few hints on how to add this piece of functionality might be implemented, consider how you might extract the actual `int` number from the course code attribute.

Additionally, you should implement a `__eq__` on `StanfordCourse`s. Two classes are equivalent if they are in the same department and have the same course code: the course title doesn't matter here.

In [6]:
class StanfordCSCourse(StanfordCourse):
    def __init__(self, department, code, title, students={}, recorded=False):
        super().__init__(department, code, title, students)
        self.is_recorded = recorded

    def __le__(self, other):
        return (self.code <= other.code)

    def __lt__(self, other):
        return (self.code < other.code)

    def __eq__(self, other):
        return (self.code == other.code)

    def __gt__(self, other):
        return (self.code > other.code)

    def __ne__(self, other):
        return (self.code != other.code)

    def __ge__(self, other):
        return (self.code >= other.code)


a = StanfordCourse("CS", "106A", "Programming Methodology")
b = StanfordCourse("CS", "106A", "Programming Abstractions")
x = StanfordCSCourse("CS", "106A", "Programming Abstractions", recorded=True)

print(a.code)
print(b.code)
print(x.is_recorded)

print(type(a))
print(isinstance(a, StanfordCourse))  # True
print(isinstance(b, StanfordCourse))  # True
print(isinstance(x, StanfordCourse))  # True
print(isinstance(x, StanfordCSCourse))  # True
print(type(a) == type(b))  # True
print(type(b) == type(x))  # False
print(a == b)  # False
print(b == x)  # False

106A
106A
True
<class '__main__.StanfordCourse'>
True
True
True
True
True
False
False
True


#### Sorting

Now that we've written a `__le__` method and an `__eq__` method, we've implemented everything we need to speak about an "ordering" of `StanfordCourse`s. Using the [`functools.total_ordering` decorator](https://docs.python.org/3/library/functools.html#functools.total_ordering), decorate the class so that all of the comparison methods are implemented. You should be able to run

In [7]:
cs106a = StanfordCourse("CS", "106A", "Programming Methodology")
cs106b = StanfordCSCourse("CS", "106B", "Programming Abstractions")
cs107 = StanfordCSCourse("CS", "107", "Computer Organzation and Systems")
cs110 = StanfordCSCourse("CS", "110", "Principles of Computer Systems")
print(cs110 > cs106b) 
print(cs107 > cs110)  

print()

courses = [cs110, cs106a, cs107, cs106b]
for x in courses: print(x.code, end=" ")  
courses.sort()
print()
for x in courses: print(x.code, end=" ")  

True
False

110 106A 107 106B 
106A 106B 107 110 

### Catalog

Implement a class called `CourseCatalog` that is constructed from a list of `StanfordCourse`s. Write a method for the `CourseCatalog` which returns a list of courses in a given department. Additionally, write a method for `CourseCatalog` that returns all courses that contain a given piece of search text in their title.

Feel free to implement any other interesting methods you'd like.

In [8]:
class CourseCatalog:
    def __init__(self, courses):
        self.courses = courses
       
    def courses_by_department(self, department_name):
        lst = []
        for elemt in self.courses:
            if elemt.department == department_name:
                lst.append(elemt)
        return lst
        
    def courses_by_search_term(self, search_snippet):
        lst = []
        for elemt in self.courses:
            if search_snippet in elemt.title:
                lst.append(elemt)
        return lst

In [9]:
cs106a = StanfordCSCourse("CS", "106A", "Programming Methodology")
cs106b = StanfordCSCourse("CS", "106B", "Programming Abstractions")
cs107 = StanfordCSCourse("CS", "107", "Computer Organzation and Systems")
cs110 = StanfordCSCourse("CS", "110", "Principles of Computer Systems")
ts100 = StanfordCSCourse("TS", "100", "Logic of Programming")
courses = [cs110, cs106a, cs107, cs106b, ts100]
x= CourseCatalog(courses)

In [10]:
resp = x.courses_by_department("CS")
for i in resp:
    print(i.department, i.code, i.title)

CS 110 Principles of Computer Systems
CS 106A Programming Methodology
CS 107 Computer Organzation and Systems
CS 106B Programming Abstractions


In [11]:
resp = x.courses_by_department("TS")
for i in resp:
    print(i.department, i.code, i.title)

TS 100 Logic of Programming


In [12]:
resp = x.courses_by_search_term("Logic")
for i in resp:
    print(i.department,i.code, i.title)

TS 100 Logic of Programming


### Remove (challenge)

Implement a method on a `TimedKVStore` to `remove(key)` that takes a key and removes that entire key from the key-value store.

Write another `remove(key, time)` method that takes a key and removes all memory of values before that time method.

## Inheritance

Consider the following code:

```Python
"""Examples of Single Inheritance"""
class Transportation:
    wheels = 0

    def __init__(self):
        self.wheels = -1

    def travel_one(self):
        print("Travelling on generic transportation")

    def travel(self, distance):
        for _ in range(distance):
            self.travel_one()

    def is_auto(self):
        return self.wheels == 4

class Bike(Transportation):

    def travel_one(self):
        print("Biking one mile")

class Car(Transportation):
    wheels = 4

    def travel_one(self):
        print("Driving one mile")

    def make_sound(self):
        print("VROOM")

class Ferrari(Car):
    pass

t = Transportation()
b = Bike()
c = Car()
f = Ferrari()
```

Predict the outcome of each of the following lines of code.

```Python
isinstance(t, Transportation)

isinstance(b, Bike)
isinstance(b, Transportation)
isinstance(b, Car)
isinstance(b, t)

isinstance(c, Car)
isinstance(c, Transportation)

isinstance(f, Ferrari)
isinstance(f, Car)
isinstance(f, Transportation)

issubclass(Bike, Transportation)
issubclass(Car, Transportation)
issubclass(Ferrari, Car)
issubclass(Ferrari, Transportation)
issubclass(Transportation, Transportation)

b.travel(5)
c.is_auto()
f.is_auto()
b.is_auto()
b.make_sound()
c.travel(10)
f.travel(4)
```

In [13]:
class Transportation:
    wheels = 0

    def __init__(self):
        self.wheels = -1

    def travel_one(self):
        print("Travelling on generic transportation")

    def travel(self, distance):
        for _ in range(distance):
            self.travel_one()

    def is_auto(self):
        return self.wheels == 4

class Bike(Transportation):

    def travel_one(self):
        print("Biking one mile")

class Car(Transportation):
    wheels = 4

    def travel_one(self):
        print("Driving one mile")

    def make_sound(self):
        print("VROOM")

class Ferrari(Car):
    pass

t = Transportation()
b = Bike()
c = Car()
f = Ferrari()

In [14]:
print(isinstance(t, Transportation))

print(isinstance(b, Bike))
print(isinstance(b, Transportation))
print(isinstance(b, Car))
print(isinstance(b, type(Car)))

print(isinstance(c, Car))
print(isinstance(c, Transportation))

print(isinstance(f, Ferrari))
print(isinstance(f, Car))
print(isinstance(f, Transportation))

print(issubclass(Bike, Transportation))
print(issubclass(Car, Transportation))
print(issubclass(Ferrari, Car))
print(issubclass(Ferrari, Transportation))
print(issubclass(Transportation, Transportation))

b.travel(5)
print(c.is_auto())
print(f.is_auto())
print(b.is_auto())
# b.make_sound()
c.travel(10)
f.travel(4)

True
True
True
False
False
True
True
True
True
True
True
True
True
True
True
Biking one mile
Biking one mile
Biking one mile
Biking one mile
Biking one mile
False
False
False
Driving one mile
Driving one mile
Driving one mile
Driving one mile
Driving one mile
Driving one mile
Driving one mile
Driving one mile
Driving one mile
Driving one mile
Driving one mile
Driving one mile
Driving one mile
Driving one mile


## Magic Methods

### Reading

Python provides an enormous number of special methods that a class can override to interoperator with builtin Python operations. You can skim through an [approximate visual list](http://diveintopython3.problemsolving.io/special-method-names.html) from Dive into Python3, or a [more verbose explanation](https://rszalski.github.io/magicmethods/), or the [complete Python documentation](https://docs.python.org/3/reference/datamodel.html#specialnames) on special methods. Fair warning, there are a lot of them, so it's probably better to skim than to really take a deep dive, unless you're loving this stuff.

### Writing (Polynomial Class)

We will write a `Polynomial` class that acts like a number. As a a reminder, a [polynomial](https://en.wikipedia.org/wiki/Polynomial) is a mathematical object that looks like $1 + x + x^2$ or $4 - 10x + x^3$ or $-4 - 2x^{10}$. A mathematical polynomial can be evaluated at a given value of $x$. For example, if $f(x) = 1 + x + x^2$, then $f(5) = 1 + 5 + 5^2 = 1 + 5 + 25 = 31$.

Polynomials are also added componentwise: If $f(x) = 1 + 4x + 4x^3$ and $g(x) = 2 + 3x^2 + 5x^3$, then $(f + g)(x) = (1 + 2) + 4x + 3x^2 + (4 + 5)x^3 = 3 + 4 + 3x^2 + 9x^3$.

Construct a polynomial with a variadic list of coefficients: the zeroth argument is the coordinate of the $x^0$'s place, the first argument is the coordinate of the $x^1$'s place, and so on. For example, `f = Polynomial(1, 3, 5)` should construct a `Polynomial` representing $1 + 3x + 5x^2$.

You will need to override the addition special method (`__add__`) and the callable special method (`__call__`).

You should be able to emulate the following code:

```Python
f = Polynomial(1, 5, 10)
g = Polynomial(1, 3, 5)

print(f(5))  # => Invokes `f.__call__(5)`
print(g(2))  # => Invokes `g.__call__(2)`

h = f + g    # => Invokes `f.__add__(g)`
print(h(3))  # => Invokes `h.__call__(3)`
```

Lastly, implement a method to convert a `Polynomial` to an informal string representation. For example, the polynomial `Polynomial(1, 3, 5)` should be represented by the string `"1 * x^0 + 3 * x^1 + 5 * x^2"`.

In [15]:
class Polynomial:
    def __init__(self, *coeffs):
        self.coeffs = coeffs
    
    def __call__(self, x):
        res = 0
        for index, coeff in enumerate(self.coeffs):
            res += coeff * x ** index
        return res
    def __add__(self, other):
        """Implement `self + other`."""
        if len(self.coeffs) > len(other.coeffs):
            result_coeff = self.coeffs[:]
            for i in range(len(other.coeffs)):
                result_coeff[i] += other.coeffs[i]
        else:
            result_coeff = other.coeffs[:]
            for i in range(len(self.coeffs)):
                result_coeff[i] += self.coeffs[i]
        return Polynomial(result_coeff)
    
    def __str__(self):
        """Implement `str(x)`."""
        res = ""
        for i in range(len(self.coeffs)-1, -1, -1):
            res +=  str(self.coeffs[i]) + "x^" + str(i) + " + "
        if res.endswith(" + "):
            res = res[:-3]
        return res

In [16]:
f = Polynomial(1, 5, 10)
print(f(1))
print(f)

16
10x^2 + 5x^1 + 1x^0


In [17]:
f = Polynomial(1,2,3)
print(f(5))
print(f)

86
3x^2 + 2x^1 + 1x^0


In [18]:
f = Polynomial(1, 5, 10)
g = Polynomial(1,2,3)
f(1)+g(1)

22

In [19]:
f = Polynomial(2, 2, 2)
g = Polynomial(2, 2,2)
f(1)+g(1)

12

## Exceptions

### Reading

Skim over [Python's documentation on built-in exceptions](https://docs.python.org/3.4/library/exceptions.html).

### `try`/`except`/`else`/`finally`

Python provides `try` and `except` blocks , similar to other languages' `try` and `catch` blocks, for basic exceptional control flow.

#### `get_age`

Write a function `get_age` that asks a user for their age, which must be a positive integer between 0 and 123 inclusive (the oldest human recorded, Jeanna Clement, died at the age of 122). If the user enters something that's not an integer, you should reprompt them. However, if they enter an integer and it's out of range, you should raise an exception. That is, you should keep reprompting them until they enter something that can be converted into an integer, and then return that number if it's in range, and raise an exception otherwise. Two sample runs are shown below

```
# (Call 1)
How old are you? ABC
Invalid integer input.
How old are you? -4.5
Invalid integer input.
How old are you? 36
# returns 36

# (Call 2)
How old are you? XYZ
Invalid integer input.
How old are you? 128
# raises some exception
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
ValueError: Age 128 out of range
```


In [21]:
def get_age(min_age=0, max_age=123):
    try:
        a = int(input("How old are you? "))
        if(a < min_age or a > max_age):
            print(a, ' out of range')
        else:
            print("You are {0} years old".format(a))
    except ValueError:
        print("Invalid integer input.")

In [22]:
get_age()

How old are you? 18
You are 18 years old


In [23]:
get_age()

How old are you? 25
You are 25 years old


In [24]:
get_age()

How old are you? 71
You are 71 years old


In [25]:
get_age()

How old are you? 80
You are 80 years old


In [26]:
get_age()

How old are you? 01
You are 1 years old


In [27]:
get_age()

How old are you? 10
You are 10 years old


### Custom Exceptions

Write a custom exception class called `OutOfRangeError` that inherits from `ValueError` which indicates that a given value is outside of an acceptable range. What does this class definition look like? What is the body of the class?

``` 
# Implement OutOfRangeError
```

Rewrite your code in `get_age` to use this custom exception. Do you need to change any other code?


In [28]:
class OutOfRangeError(ValueError):
    pass

def get_age():
    try:
        a = int(input("How old are you? "))
        if (a < 0 or a > 123):
            myError = OutOfRangeError()
            raise myError
        print(a)
    except myError:
        print(a, ' out of range')
    except Exception:
        print("Invalid integer input.")
    else:
        print("In else.")
    finally:
        print("In finally:")

get_age()

How old are you? 19
19
In else.
In finally:


### Using `else` and `finally`

Rewrite your `get_age` function to use the `else` block, and optionally the `finally` block. As is consistent with general style guidelines, try to keep the `try` block as short as possible, containing just the code that might raise the exception you're trying to catch.


### Reraising

Consider the following code:

```Python
try:
    print("in try")
    # (A)
except Exception as exc:
    print("in except")
    # (B)
else:
    print("in else")
    # (C)
finally:
    print("in finally")
    # (D)
```

We're going to add some errors to this code block, which currently prints out

```
in try
in else
in finally
```

For each of the labelled locations `(A), (B), (C), (D)`, which statements print out if we `raise Exception()` at that position? Run the code to test your hypotheses.

In [29]:
# Case (A)
try:
    print("Try")
    raise Exception('An on-purpose exception.')
except Exception as exc:
    print("Except")
else:
    print("Else")
finally:
    print("Finally")

Try
Except
Finally


In [30]:
# Case (B)
try:
    print("Try")
except Exception as exc:
    print("Except")
    raise Exception('An on-purpose exception.')
else:
    print("Else")
finally:
    print("Finally")

Try
Else
Finally


In [31]:
# Case (C)
try:
    print("Try")
except Exception as exc:
    print("Except")
else:
    print("Else")
    raise Exception('An on-purpose exception.')
finally:
    print("Finally")

Try
Else
Finally


Exception: An on-purpose exception.

In [32]:
# Case (D)
try:
    print("Try")
except Exception as exc:
    print("Except")
else:
    print("Else")
finally:
    print("Finally")
    raise Exception('An on-purpose exception.')

Try
Else
Finally


Exception: An on-purpose exception.

In [33]:
# Case (AB)
try:
    print("Try")
    raise Exception('An on-purpose exception.')
except Exception as exc:
    print("Except")
    raise Exception('Another on-purpose exception.')
else:
    print("Else")
finally:
    print("Finally")

Try
Except
Finally


Exception: Another on-purpose exception.

In [34]:
# Case (AC)
try:
    print("Try")
    raise Exception('An on-purpose exception.')
except Exception as exc:
    print("Except")
else:
    print("Else")
    raise Exception('Another on-purpose exception.')
finally:
    print("Finally")

Try
Except
Finally


In [35]:
# Case (AD)
try:
    print("Try")
    raise Exception('An on-purpose exception.')
except Exception as exc:
    print("Except")
else:
    print("Else")
finally:
    print("Finally")
    raise Exception('Another on-purpose exception.')

Try
Except
Finally


Exception: Another on-purpose exception.

## Done Early?

Take a deep breath. Whatever you're working on, you can do it!

> With <3 by @sredmond